In [ ]:
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.1 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from pyvi import ViTokenizer, ViPosTagger
import string
import os
from tqdm import tqdm
import numpy as np
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/Text_Generator

/content/drive/My Drive/Colab Notebooks/Text_Generator


In [ ]:
%ls -la

total 233
drwx------ 2 root root  4096 Apr 14 03:53 data/
-rw------- 1 root root 63670 Apr 26 11:00 data_processing.ipynb
drwx------ 2 root root  4096 Apr 14 04:17 model/
-rw------- 1 root root  4482 Apr 26 04:26 scrape_news.ipynb
-rw------- 1 root root 21702 May  3 03:54 search_generate_text.ipynb
drwx------ 2 root root  4096 Apr 26 03:59 src/
drwx------ 2 root root  4096 Apr 14 09:19 temp/
-rw------- 1 root root 53276 May  3 03:53 text_generator.ipynb
-rw------- 1 root root 77160 Apr 26 12:23 word_embedding.ipynb


## **Load pre-trained model**

In [ ]:
with open('./model/tokenizer.pkl', 'rb') as f:
  tokenizer = pickle.load(f)

with open('./model/sequences_digit.pkl', 'rb') as f:
  sequences_digit = pickle.load(f)

model = tf.keras.models.load_model('./model/51_acc_language_model.h5')

## **Data processing functions**

In [ ]:
def clean_document(doc):
  doc = ViTokenizer.tokenize(doc) #Pyvi Vitokenizer library
  doc = doc.lower() #Lower
  tokens = doc.split() #Split in_to words
  table = str.maketrans('', '', string.punctuation.replace("_", "")) #Remove all punctuation
  tokens = [w.translate(table) for w in tokens]
  tokens = [word for word in tokens if word]

  return tokens

In [ ]:
def preprocess_input(doc):
  tokens = clean_document(doc)
  tokens = tokenizer.texts_to_sequences(tokens)
  tokens = tf.keras.preprocessing.sequence.pad_sequences([tokens], maxlen=50, truncating='pre')

  return np.reshape(tokens, (1,50))

In [ ]:
def top_n_words(text_input, top_n=3):
  tokens = preprocess_input(text_input)
  predictions = model.predict(tokens)[0]

  # Lấy top k dự đoán cao nhất
  top_indices = np.argpartition(predictions, -top_n)[-top_n:]

  top_words = []
  for index in top_indices:
    for word, idx in tokenizer.word_index.items():
      if idx == index:
        top_words.append(word)
        break

  return top_words

In [ ]:
def generate_sentences(text_input, n_words):
  tokens = preprocess_input(text_input)
  generated_sentences = []
  for _ in range(n_words):
    next_digit = np.argmax(model.predict(tokens, verbose=0))
    tokens = np.append(tokens, next_digit)
    tokens = np.delete(tokens, 0)
    tokens = np.reshape(tokens, (1, 50))

  # Mapping to text
  tokens = np.reshape(tokens, (50))
  # print(tokens)
  out_word = []
  for token in tokens:
    for word, index in tokenizer.word_index.items():
      if index == token:
        out_word.append(word)
        break

  return ' '.join(out_word)


## **Generate a sentence with n words**

In [ ]:
text = generate_sentences('phó_giám_đốc phụ_trách kinh_doanh', 20)
print(text)

phó_giám_đốc phụ_trách kinh_doanh ngoại_hối một ngân_hàng biết muốn vội_vàng kết_hôn muốn một đứa con tuy_nhiên cô muốn ân_hận khó_khăn đường con_cái lấy chồng


## **Generate top 3 possible texts**

In [ ]:
# Khởi tạo danh sách để lưu các câu
generated_sentences = []

# Với mỗi text_input, gọi top_n_words để lấy 3 từ có khả năng.
text_input = "phó_giám_đốc phụ_trách kinh_doanh"
top_words = top_n_words(text_input, top_n=3)

# Với mỗi từ trong top_words, tạo ra một câu và lưu vào danh sách generated_sentences
for word in top_words:
  new_text_input = text_input + " " + word
  generated_sentence = generate_sentences(new_text_input, n_words=20)
  generated_sentences.append(generated_sentence)

# In ra các câu đã được tạo ra
for i, sentence in enumerate(generated_sentences, start=1):
  print("Generated sentence", i, ":", sentence)

1/1 [==============================] - 0s 87ms/step
Generated sentence 1 : phó_giám_đốc phụ_trách kinh_doanh đắt_giá nhau mời một hơn tiếp_theo an rạn bụng chồng áp tai bụng kể con nghe chồng viện khám định_kỳ suốt
Generated sentence 2 : phó_giám_đốc phụ_trách kinh_doanh lễ_đường một sinh bạn_đời trong hôn_nhân ảnh_hưởng tuổi mạnh_mẽ biết người trẻ ngày_nay dịch_vụ hạnh_phúc chồng tôn bảo mạng xã_hội buồn
Generated sentence 3 : phó_giám_đốc phụ_trách kinh_doanh ngoại_hối một ngân_hàng biết muốn vội_vàng kết_hôn muốn một đứa con tuy_nhiên cô muốn ân_hận khó_khăn đường con_cái lấy chồng muộn


## **Evaluate the result**

In [ ]:
# test_loss, test_acc = model.evaluate(X, y, verbose=1)

In [ ]:
# !pip install sequence-evaluate
# !pip install rouge
# !pip install sentence-transformers

In [ ]:
# from seq_eval import SeqEval
# evaluator = SeqEval()

In [ ]:
# scores = evaluator.evaluate(candidates, references, verbose=True)